## Simple Mission

This notebook serves to check the ardupilot installation

In [5]:
import os
import subprocess
import time

from pymavlink import mavutil  # type: ignore
from pymavlink.mavwp import MAVWPLoader  # type: ignore

from config import ARDUPILOT_VEHICLE_PATH, LOGS_PATH, VEH_PARAMS_PATH
from helpers import clean
from helpers.mavlink import MAVConnection, MissionResult, NavCommand
from plan import State
from plan.planner import Plan

clean()

# TODO Add annotation for static checking

## Launch Copter (ardupilot)

In [6]:
#This must agrre with first waypoint in mission.waypoint
spawn_str= '47.397742,8.545594,0,0' 

sim_vehicle_path = os.path.expanduser(ARDUPILOT_VEHICLE_PATH)
vehicle_cmd = (
    f"python3 {sim_vehicle_path} "
    f"-v ArduCopter "
    f"-I0 --sysid 1 "
    f"--no-rebuild "
    f"--use-dir={LOGS_PATH} "
    f"--add-param-file {VEH_PARAMS_PATH} "
    f"--no-mavproxy "
    f'--custom-location={spawn_str}'
)


# Open a new terminal and run the command
subprocess.Popen(["gnome-terminal","--title=planner","--", 
                  "bash", "-c", f"{vehicle_cmd}; exec bash"])

<Popen: returncode: None args: ['gnome-terminal', '--title=planner', '--', '...>

## Connect to the vehicle

In [7]:
time.sleep(1)
conn: MAVConnection = mavutil.mavlink_connection("tcp:127.0.0.1:5760") # type: ignore 14550 
conn.wait_heartbeat() 
print("✅ Heartbeat received")


✅ Heartbeat received


# 2. Load the waypoint file

In [8]:
mission = MAVWPLoader(conn.target_system,conn.target_component)
count = mission.load("mission.waypoints")  # your QGC WPL 110 file
print(f"✅ Loaded {count} waypoints")

✅ Loaded 4 waypoints


# 3. Read the mission

In [9]:
# Retrieve each mission item
for i in range(count):
    wp= mission.item(i)
    print(f"🧭 Mission[{i}] → cmd: {NavCommand(wp.command).name}, z: {wp.x}, z: {wp.y},z: {wp.z}, current: {wp.current}")


🧭 Mission[0] → cmd: MAV_CMD_NAV_WAYPOINT, z: 47.397742, z: 8.545594,z: 0.0, current: 0
🧭 Mission[1] → cmd: MAV_CMD_NAV_TAKEOFF, z: 47.397742, z: 8.545594,z: 10.0, current: 0
🧭 Mission[2] → cmd: MAV_CMD_NAV_WAYPOINT, z: 47.397787, z: 8.545594,z: 10.0, current: 0
🧭 Mission[3] → cmd: MAV_CMD_NAV_LAND, z: 47.397787, z: 8.545594,z: 0.0, current: 0


# 3. Clear any existing mission

In [10]:
conn.mav.mission_clear_all_send(conn.target_system, conn.target_component)
ack = conn.recv_match(type='MISSION_ACK', blocking=True, timeout=5)
print(f"🧹 Cleared previous mission: {ack.type if ack else 'no ack'}")
time.sleep(1)

🧹 Cleared previous mission: 0


# 4. Send new mission count

In [11]:
conn.mav.mission_count_send(conn.target_system, conn.target_component, mission.count())
print(f"📤 Sending {mission.count()} mission items")


📤 Sending 4 mission items


# 5. Send each item in response to MISSION_REQUEST

In [12]:
for i in range(mission.count()):
    msg = conn.recv_match(type='MISSION_REQUEST', blocking=True, timeout=5)
    if not msg or msg.seq != i:
        raise RuntimeError(f"❌ Unexpected mission request: {msg}")
    conn.mav.send(mission.wp(i))
    print(f"✅ Sent mission item {i}")

✅ Sent mission item 0
✅ Sent mission item 1
✅ Sent mission item 2
✅ Sent mission item 3


# 6. Wait for final MISSION_ACK

In [13]:
ack = conn.recv_match(type='MISSION_ACK', blocking=True, timeout=5)
if ack and MissionResult(ack.type) == MissionResult.MAV_MISSION_ACCEPTED:
    print("🎉 Mission upload successful!")
else:
    print(f"⚠️ Mission upload failed or timed out: {ack}")

🎉 Mission upload successful!


## Check the mission

In [14]:
# Create a mission protocol loader
wp = MAVWPLoader()


# Request list of mission items
conn.mav.mission_request_list_send(conn.target_system, conn.target_component)

# Wait for mission count
msg = conn.recv_match(type='MISSION_COUNT', blocking=True, timeout=5)
mission_count = msg.count
print(f"📦 UAV has {mission_count} mission items")

# Retrieve each mission item
for i in range(mission_count):
    conn.mav.mission_request_send(conn.target_system, conn.target_component, i)
    msg = conn.recv_match(type='MISSION_ITEM', blocking=True, timeout=5)
    wp.set(msg, i)
    print(f"🧭 Mission[{i}] → cmd: {NavCommand(msg.command).name}, z: {msg.x}, z: {msg.y},z: {msg.z}, current: {msg.current}")


📦 UAV has 4 mission items
🧭 Mission[0] → cmd: MAV_CMD_NAV_WAYPOINT, z: 0.0, z: 0.0,z: 0.0, current: 0
🧭 Mission[1] → cmd: MAV_CMD_NAV_TAKEOFF, z: 47.397743225097656, z: 8.545594215393066,z: 10.0, current: 0
🧭 Mission[2] → cmd: MAV_CMD_NAV_WAYPOINT, z: 47.39778518676758, z: 8.545594215393066,z: 10.0, current: 0
🧭 Mission[3] → cmd: MAV_CMD_NAV_LAND, z: 47.39778518676758, z: 8.545594215393066,z: 0.0, current: 0


# Execute Plan

In [15]:
plan_auto = Plan.auto()
plan_auto.bind(conn)
while plan_auto.state != State.DONE:
    plan_auto.act()

Vehicle 1: ▶️ Plan Started: 📋 
Vehicle 1: ▶️ Action Started: 🔧 PREARM
Vehicle 1: ▶️ Step Started: Check disarmed
Vehicle 1: ✅ Step Done: Check disarmed
Vehicle 1: ▶️ Step Started: Check EKF status
Vehicle 1: ✅ Step Done: Check EKF status
Vehicle 1: ▶️ Step Started: Check GPS
Vehicle 1: ✅ Step Done: Check GPS
Vehicle 1: ▶️ Step Started: Check system
Vehicle 1: ✅ Step Done: Check system
Vehicle 1: ✅ Action Done: 🔧 PREARM
Vehicle 1: ▶️ Action Started: ⚙️ MODE: GUIDED
Vehicle 1: ▶️ Step Started: Switch to GUIDED
Vehicle 1: ✅ Step Done: Switch to GUIDED
Vehicle 1: ✅ Action Done: ⚙️ MODE: GUIDED
Vehicle 1: ▶️ Action Started: 🔐 ARM
Vehicle 1: ▶️ Step Started: arm
Vehicle 1: ✅ Step Done: arm
Vehicle 1: ✅ Action Done: 🔐 ARM
Vehicle 1: ▶️ Action Started: 🚀 START_MISSION
Vehicle 1: ▶️ Step Started: start_mission
Vehicle 1: ✅ Step Done: start_mission
Vehicle 1: ✅ Action Done: 🚀 START_MISSION
Vehicle 1: ✅ Plan Done: 📋 


# Monitoring

In [ ]:
conn.mav.request_data_stream_send(
    conn.target_system,
    conn.target_component,
    mavutil.mavlink.MAV_DATA_STREAM_POSITION,  # Stream ID
    4,     # Frequency in Hz
    1      # Start (1 to enable streaming)
)


last_seq = mission.count() - 1

while True:
    msg = conn.recv_match(blocking=True, timeout=1)

    if not msg:
        print("⏳ Waiting for mission update...")
        continue

    # ✅ Reached a waypoint
    if msg.get_type() == "MISSION_ITEM_REACHED":
        print(f"📌 Reached waypoint: {msg.seq}")
        if msg.seq == last_seq:
            print("✅ Final waypoint reached")

    # ✅ UAV position
    elif msg.get_type() == "GLOBAL_POSITION_INT":
        lat = msg.lat / 1e7
        lon = msg.lon / 1e7
        alt = msg.relative_alt / 1000.0
        print(f"📍 Position: lat={lat:.7f}, lon={lon:.7f}, alt={alt:.2f} m")

    # ✅ Look for end hints in text
    elif msg.get_type() == "STATUSTEXT":
        text = msg.text.strip().lower()
        if "disarming" in text:
            print("🏁 Mission completed")
            break

📍 Position: lat=47.3977420, lon=8.5455940, alt=0.00 m
📍 Position: lat=47.3977420, lon=8.5455939, alt=0.00 m
📍 Position: lat=47.3977420, lon=8.5455939, alt=0.00 m
📍 Position: lat=47.3977420, lon=8.5455939, alt=0.00 m
📍 Position: lat=47.3977420, lon=8.5455939, alt=0.00 m
📍 Position: lat=47.3977420, lon=8.5455939, alt=0.00 m
📍 Position: lat=47.3977420, lon=8.5455939, alt=0.00 m
📍 Position: lat=47.3977420, lon=8.5455939, alt=0.00 m
📍 Position: lat=47.3977420, lon=8.5455939, alt=0.00 m
📍 Position: lat=47.3977420, lon=8.5455939, alt=0.02 m
📍 Position: lat=47.3977420, lon=8.5455939, alt=0.07 m
📍 Position: lat=47.3977420, lon=8.5455939, alt=0.17 m
📍 Position: lat=47.3977420, lon=8.5455939, alt=0.45 m
📍 Position: lat=47.3977420, lon=8.5455939, alt=0.73 m
📍 Position: lat=47.3977420, lon=8.5455939, alt=1.05 m
📍 Position: lat=47.3977420, lon=8.5455939, alt=1.44 m
📍 Position: lat=47.3977420, lon=8.5455938, alt=1.89 m
📍 Position: lat=47.3977420, lon=8.5455938, alt=2.40 m
📍 Position: lat=47.3977420, 